<a href="https://colab.research.google.com/github/iceman67/-Python/blob/master/ASOS_openAPI_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ASOS openAPI to CSV

In [ ]:
import requests

In [ ]:
def set_url(key, numOfRows, startDt, endDt, stnId, ctype):
    base = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    url =f"{base}?serviceKey={serviceKey}&pageNo=1&numOfRows={numOfRows}&dataType={ctype}&dataCd=ASOS&dateCd=DAY&startDt={startDt}&endDt={endDt}&stnIds={stnId}"

    return url


In [ ]:
# import requests

def get_contents(URL):
    # GET 메소드로 HTTP 요청을 전달함
    response = requests.get(URL)
    print ("status code : ", response.status_code)
    # 바이트를 문자열로 변환 
    result = response.content.decode('utf-8')

    return response

In [ ]:
def get_asos_info(response):
    # 문자열 JSON 형식을 딕셔너리 객체로 변환 
    #res = json.loads(result) # jon 라이브러리 사용
    res = response.json() # requests 라이브러리 사용
    items = res['response']['body']['items'] 
    item = items['item'] # [.... ] 배열 
    print (len(item))

    return item


In [ ]:
serviceKey = 'y80jnESQZu1%2B%2BKrpWpkGrnZ96%2FhiBicuIH%2F3SeO0u10CK9rglO3nqmwetj8%2BRHj%2F1NWUUis4aeGnUMk1CFUYRQ%3D%3D'
numOfRows = 400
startDt = 20180101
endDt = 20181231
stnId =108 # 지역
#URL=f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={serviceKey}&pageNo=1&numOfRows={numOfRows}&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt={startDt}&endDt={endDt}&stnIds={stnId}"
#URL

In [ ]:
URL = set_url(serviceKey, numOfRows, startDt, endDt, stnId, "JSON")
result = get_contents(URL)
item = get_asos_info(result)
# 첫번쨰 기상정보


기상정보를 확인함

In [ ]:
item[0].keys()

기상정보 선택 및 CSV 저장

In [ ]:
import csv

max_ws = 'ASOS_108.csv'
f = open(f'{max_ws}','w', encoding="cp949")
schema = ['stnId','tm', 'avgTa', 'avgPa', 'maxWs','sumRnDur', 'minRhmHrmt', 'avgRhm']

wr = csv.writer(f)

# 리스트 자료형을 활용하여 헤더를 출력함 
wr.writerow([g for g in schema])   

# 수집된 자료의 특징 자료만을 추출하여 출력을 위한 행을 만들어 출력함  
for i in range(len(item)):
     stnId = item[i]['stnId'] 
     tm = item[i]['tm'] 
     avgTa = item[i]['avgTa']
     avgPa = item[i]['avgPa']
     maxWs = item[i]['maxWs'] 
     sumRnDur = item[i]['sumRnDur']
     minRhmHrmt = item[i]['minRhmHrmt']
     avgRhm = item[i]['avgRhm']
     wr.writerow ([ stnId, tm, avgTa, avgPa, maxWs, sumRnDur, minRhmHrmt, avgRhm])

f.close()

In [ ]:
import csv

def write_cvs(fname,  schema,item):
    fname = f'{fname}.csv'
    f = open(fname,'w', encoding="cp949")
    wr = csv.writer(f)
    # 리스트 자료형을 활용하여 헤더를 출력함 
    wr.writerow([g for g in schema])  

    # 수집된 자료의 특징 자료만을 추출하여 출력을 위한 행을 만들어 출력함  
    for i in range(len(item)):
        record = []
        for j in schema:
            record.append(item[i][j])
        wr.writerow (record)
    f.close()

In [ ]:
write_cvs('108', schema, item)

In [ ]:
import pandas as pd

infile ='ASOS_108.csv'
#data = pd.read_csv(infile,names=['stnId','tm', 'avgTa', 'avgPa', 'maxWs','sumRnDur', 'minRhmHrmt'], parse_dates=['tm'], header=None)

data = pd.read_csv(infile, parse_dates=['tm'])

In [ ]:
data.head()

avgRhm가 60 보다 작으면 좋은 날씨

In [ ]:
import numpy as np

data['w_cond'] = np.where(data['avgRhm'] < 65, True, False)

In [ ]:
data['w_cond_str'] = data['w_cond'].map({True: 'Good weather', False: 'Bad weather'})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
movie_file_path = "/content/drive/MyDrive/DA/movie2018.csv"


df_movie = pd.read_csv(movie_file_path, names = ['tm', 'title', 'cnt', 'salesShare', 'salesChange'], parse_dates=['tm'])

In [ ]:
df_movie.drop('salesChange',axis=1, inplace=True)
df_movie.drop('salesShare',axis=1, inplace=True)
grouped = df_movie.groupby(df_movie['tm'], as_index=False).sum()

In [ ]:
grouped.head()

In [ ]:
merge_df = pd.merge(data,grouped, how='outer',on='tm')

In [ ]:
merge_df.drop('stnId',axis=1, inplace=True)
merge_df.head()

sumRnDur과 avgRhm은 양의 상관관계를 가짐

In [ ]:
data_corr = merge_df.corr()
data_corr

In [ ]:
wgroup = merge_df.groupby(merge_df['w_cond'], as_index=False).sum()
wgroup['w_cond_str'] = wgroup['w_cond'].map({True: 'Good weather', False: 'Bad weather'}) 
summary = wgroup[ ['w_cond_str', 'cnt']]


In [ ]:
summary

In [ ]:
plot = summary.plot.pie(y='cnt', figsize=(5, 5))

In [ ]:
import matplotlib.pyplot as plt

plt.bar ( wgroup['w_cond_str'], wgroup['cnt'])